# <p class='alert alert-info'>Proyecto final</p>

## <p class='alert alert-info'>Importación de librerías</p>

In [1]:
###
# Importaciones
###
# Data
import pandas as pd
import numpy as np
# Request
import requests
# Scrapper
from bs4 import BeautifulSoup
# Mapas
import geocoder
from geopy.geocoders import Nominatim
import folium

ModuleNotFoundError: No module named 'geocoder'

## <p class='alert alert-info'>Obtención de datos</p>

In [ ]:
# Obtener la página web
URL = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = requests.get(URL)
soup = BeautifulSoup(page.content, "html.parser")
tables = soup.find_all("table") # Obtenemos los elementos del texto html que contengan la etiqueta table

In [ ]:
toronto_postal_codes = pd.DataFrame({'PostalCode':[],
                                     'Borough':[],
                                     'Neighborhood':[]}) # Declaramos el DF
for index,table in enumerate(tables): # Si hay más elemento en el html que sean table, iteramos
    for td in table.find_all("td"): # De la etiqueta table buscamos las etiquetas td
        text = td.find("span").get_text() # Obtenemos el texto que esta dentro de span
        if "Not assigned" not in text: # Evitamos los no asignados
            postal_code = td.find('b').get_text()
            borough = text.split('(')[0]
            text = text.split('(')[1].replace(')','')# Limpiamos la cadena quitando los ()
            lista_vecindarios = text.split('/') # Romper la cadena
            if len(lista_vecindarios) > 1:
                neighborhoods = ', '.join(lista_vecindarios)
            else:
                neighborhoods = lista_vecindarios[0]
            toronto_postal_codes = toronto_postal_codes.append({'PostalCode':postal_code,
                                                                'Borough':borough,
                                                                'Neighborhood':neighborhoods},
                                                               ignore_index=True) # Guardamos el DF
    break # Rompemos a la primer iteración de los elementos table, ya que el primero es el que nos interesas

## <p class='alert alert-info'>DataFrame resultante</p>

In [ ]:
toronto_postal_codes.sort_values(by='PostalCode')

## <p class='alert alert-info'>DataFrame resultante</p>

In [ ]:
# Enriqueciendo el DF con la latitud y longitud.
# Metodo API
"""lista_latidud = []
lista_longitud = []
for row in toronto_postal_codes.iterrows():
    print(row['PostalCode'])
    g = geocoder.google(f"{row['PostalCode']} , Toronto, Ontario")
    lat_lng_coords = g.latlng
    lista_latidud.append(lat_lng_coords[0])
    lista_longitud.append(lat_lng_coords[1])
toronto_postal_codes['Latitude'] = lista_latidud
toronto_postal_codes['Longitude'] = lista_longitud"""
# Metodo archivo
coordenadas = pd.read_csv("Geospatial_Coordinates.csv")
toronto_postal_codes = pd.merge(toronto_postal_codes,coordenadas,left_on='PostalCode',right_on='Postal Code',how='inner')
toronto_postal_codes

## <p class='alert alert-info'>Mapas de Toronto</p>

In [ ]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto Ontario are {}, {}.'.format(latitude, longitude))

In [ ]:
# crear un mapa de Nueva York utilizando los valores de latitud y longitud
map_canada = folium.Map(location=[latitude, longitude], zoom_start=10)

# añadir marcadores al mapa
for lat, lng, borough, neighborhood in zip(toronto_postal_codes['Latitude'], toronto_postal_codes['Longitude'], toronto_postal_codes['Borough'], toronto_postal_codes['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_canada)  
    
map_newyork